In [26]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer

# Import Data

In [27]:
df_tracks = pd.read_csv("C://Users//chris//spotify_music//data//spotify_songs.csv")
df_tracks.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

In [28]:
df_tracks = df_tracks.rename(columns={"track_album_id": "album_id", "track_album_name": "album"}, errors="raise")
df_tracks = df_tracks.rename(columns={"track_artist": "artist", "playlist_genre": "genre", "track_popularity": "popularity"}, errors="raise")

# Evaluation
We leverge Top-N accuracy as metric by comparing the list of top recommendations (tracks) provided to a playlist by the recommender to to the items (tracks) that actually present in the test playlist. The algorithm is as follow:
- For each test playlist:
  - For each track in the test playlist:
    - Sample n other tracks that are never present in the test playlist
    - Use the recommender model to produce a ranked list of recommended tracks, from a set composed of one real track and   
      others non-exist tracks 
    - Compute the Top-N accuracy metrics(recall) for this test playlist (Is the real track along the Top-N ranked items?)

In [73]:
import random
from sklearn.model_selection import train_test_split

def evaluate(df_tracks,model,N=100):
    
    playlists = df_tracks["playlist_id"].unique()
                          
    playlist_metric = list()
    
    total = 0
                          
    for _,playlist in enumerate(playlists):
                                           
        track_id = df_tracks[df_tracks["playlist_id"]==playlist]["track_id"].tolist()
      
        non_track_id = df_tracks[df_tracks["playlist_id"] != playlist]["track_id"].tolist()
        
        
        if len(track_id)<10:
            
           continue
        else:
            
           total += 1
      
        train,test = train_test_split(track_id, test_size = 0.2, random_state=0)
                          
        recommend_list = model.recommend(playlist,False)
                          
        recommend_ids = list(list(zip(*recommend_list))[0])
                          
        d = {k:v for v,k in enumerate(recommend_ids)}
        
        hit_5,hit_10,hit_20 = 0.0,0.0,0.0
                          
        for track in test:
      
            mylist = [track]
        
            N = 100
                          
            non_track_ids = random.sample(non_track_id,N)
                          
            mylist.extend(non_track_ids)
                          
            res_list = [d[elm] for elm in mylist]
                          
            res_list = sorted(res_list)
                          
            res_list = [recommend_ids[index] for index in res_list]
            
            if track in res_list[:5]:
                          
               hit_5 += 1
                          
            if track in res_list[:10]:
                          
               hit_10 += 1
                          
            if track in res_list[:20]:
                          
               hit_20 += 1
            
                          
        hit_5 = hit_5 / len(test)
                          
        hit_10 = hit_10 / len(test)
                          
        hit_20 = hit_20 / len(test)
                          
        playlist_metric.append((hit_5,hit_10,hit_20,len(test)))
      
    recall_5 = sum(list(list(zip(*playlist_metric))[0])) / total
                          
    recall_10 = sum(list(list(zip(*playlist_metric))[1])) / total
                        
    recall_20 = sum(list(list(zip(*playlist_metric))[2])) / total
                          
    playlist_metrics = {
                            'recall@5': recall_5,
                            'recall@10': recall_10,
                            'recall@20': recall_20
                     
                        }
    
    print(playlist_metrics)

# Content-based Recommender

   Content-Based recommender system tries to guess the features or behavior of a user given the item’s features that he or 
   she interacts positively to. During recommendation, several similarity metrics are calculated from the item’s feature 
   vectors and the user’s preferred feature vectors from his/her previous records. Then, the top few are recommended.
   
   We leverage an information retrieval method named TF-IDF. This technique embeds unstructured text into a 
   vector structure, where each word is represented by a position in the vector, and the value determines how relevant a 
   given word is for an article. We can then compute the cosine similarity between the user vector and the initial sparse 
   matrix (all songs).

# TF-IDF

In [74]:
def get_text_description(df):

      df["full_description"] = df["artist"] + df["playlist_name"] + df["album"] + df["track_name"] + df["genre"]
        
      df["full_description"] = df["artist"] + df["album"] + df["track_name"] + df["genre"]
      
      vectorizer = TfidfVectorizer( analyzer = 'word',
                                    ngram_range = (1,2),
                                    min_df = 0.003,
                                    max_df = 0.5,
                                    max_features=5000)
      
      tfid_matrix = vectorizer.fit_transform(df["full_description"].values.astype('U'))
      
      feat = vectorizer.get_feature_names_out()
      
      return tfid_matrix,feat

In [75]:
tfid_matrix, feat = get_text_description(df_tracks)
tfid_matrix

<32833x164 sparse matrix of type '<class 'numpy.float64'>'
	with 56364 stored elements in Compressed Sparse Row format>

In [76]:
def construct_profile(df,tfid_matrix):
      
      playlist_profile = {}
      
      for playlistid in df["playlist_id"].unique():
      
          mylist = []
         
          index = df[df["playlist_id"] == playlistid].index
        
          for i in index:
                
              mylist.append(tfid_matrix[i:i+1])
    
              profile_sum = np.sum(mylist,axis=0)
       
              profile_sum = normalize(profile_sum)
     
              playlist_profile[playlistid] = profile_sum
      
      return playlist_profile    

In [77]:
playlist_profile = construct_profile(df_tracks,tfid_matrix)
len(playlist_profile)

471

# Track Content - Based Recommender
We then incorporate the TF-IDF based playlists profile into our content based Collaborative Recommender.

In [78]:
class Track_Content_CollaborativeRecommender:
      
      def __init__(self,tfid_matrix,playlist_profile,df_tracks):
     
          self.tfid_matrix = tfid_matrix
         
          self.playlist_profile = playlist_profile
      
          self.df_tracks = df_tracks
         
      def find_similar_tracks(self,playlist_id):
        
          similarity_measure = cosine_similarity(self.playlist_profile[playlist_id],tfid_matrix)
                                                 
          similar_indices = similarity_measure.argsort().flatten()
        
          similar_tracks = sorted([(self.df_tracks['track_id'].tolist()[i], similarity_measure[0,i]) for i in similar_indices], key=lambda x: -x[1])
     
    
          return similar_tracks
    
    
      def recommend(self,playlist_id,is_exempt = True):
      
          if not is_exempt:
            
             exempt_id = []
     
          else:
            
             exempt_id = self.df_tracks[self.df_tracks["playlist_id"] == playlist_id]["track_id"].tolist()
     
          similar_tracks = self.find_similar_tracks(playlist_id)
        
          recommend_track = list()
            
          for track in similar_tracks:
       
              if track[0] not in exempt_id:
      
                 recommend_track.append((track[0],track[1]))
            
          return recommend_track

In [79]:
track_collaborate = Track_Content_CollaborativeRecommender(tfid_matrix,playlist_profile,df_tracks)

In [80]:
evaluate(df_tracks,track_collaborate,N=100)

{'recall@5': 0.2936288393765006, 'recall@10': 0.40874643415545653, 'recall@20': 0.5268713513674977}


In [81]:
recommend_list = track_collaborate.recommend("37i9dQZF1DX2ENAPP1Tyed")

In [82]:
recommend_list = pd.DataFrame(recommend_list,columns=["track_id","intensity"])
recommendations = pd.merge(recommend_list,df_tracks.drop_duplicates(subset='track_id', keep="first"), how='left', on='track_id')
recommendations[['track_id', 'track_name', 'artist', 'album', 'intensity']].head()


,track_id,track_name,artist,album,intensity
0,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",Flower Boy,0.717705
1,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",Flower Boy,0.717705
2,1m7XbKhOvJst478ms8yUrr,Spin With You (feat. Jeremy Zucker),Emma Sameth,Spin With You (feat. Jeremy Zucker),0.711192
3,6Pgkp4qUoTmJIPn7ReaGxL,Without You (feat. Sandro Cavazza),Avicii,AVĪCI (01),0.675595
4,6WbADFqMvR8N5u0BvtsWQE,Without You (feat. Sandro Cavazza),Avicii,AVĪCI (01),0.675595


# Latent Factors Model
Latent Factor models are a state of the art methodology for model-based collaborative filtering. The basic assumption is that there exist an unknown low-dimensional representation of users and items where user-item affinity can be modeled accurately. For example, the rating that a user gives to a movie might be assumed to depend on few implicit factors such as the user’s taste across various movie genres. If we choose a very large factor, factorisation will be more accurate as more information of the original matrix are captured. However, reducing the number of factors can increases the model generalisation which is beneficial for prediction. 
We leverage one of the latent factors factorization method called Singular Value Decomposition (SVD) as follow:


In [83]:
df_tracks["exist"] = 1
df_svd = df_tracks.pivot_table(values = "exist", index= "playlist_id", columns = "track_id", aggfunc='sum').fillna(0)

In [84]:
df_svd

track_id,0017A6SJgTbfQVU2EtsPNo,002xjHwzEx66OWFV2IP9dk,004s3t0ONYlzxII9PLgU6z,008MceT31RotUANsKuzy3L,008rk8F6ZxspZT4bUlkIQG,00EPIEnX1JFjff8sC6bccd,00FR9VQ0uzF4NNxVKKiMz2,00FROhC5g4iJdax5US8jRr,00GfGwzlSB8DoA0cDP2Eit,00Gu3RMpDW2vO9PjlMVFDL,...,7zsXy7vlHdItvUSH8EwQss,7ztN8tfXYaC8LBRyLZJLA1,7ztcWZ0EB6akSzlb8BUaqG,7zwOSMqWghSCfZrVo5r5nR,7zxLkZbUxITHabPzGN8Xgc,7zxRMhXxJMQCeDDg0rKAVo,7zyLObYw4QUKQDyZOb4J0Y,7zycSpvjDcqh6YT1FEl2kY,7zye9v6B785eFWEFYs13C2,7zzZmpw8L66ZPjH1M6qmOs
playlist_id,,,,,,,,,,,,,,,,,,,,,
0275i1VNfBnsNbPl0QIBpG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
03qQtbNHoJuFezRu2CnLuF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
03sDEv7FN58Mb9CJOs1Tgn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06zrBJ5cts5aemZmqe80J7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07SNJ4MwYba9wwmzrbjmYi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7sq6nuruoMyDhEWkX2oYOg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7tkgK1tm9hYkWp7EFyOcAr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7vJOXFe40axY7qS39vGDyH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
def construct_svd_df(df,playlist_index):
      
    columns_name = df.columns
     
    df = df.values
    
    df1 = csr_matrix(df)
    
    df1 = df1.asfptype()
    
    U, s, V = svds(df1,k = 50)
   
    s = np.diag(s)
    
    U_ = np.dot(U,s)
    
    svd_playlists_profile = np.dot(U_,V)
      
    print(svd_playlists_profile.shape)
    
    svd_profile = (svd_playlists_profile - svd_playlists_profile.min())/(svd_playlists_profile.max() - svd_playlists_profile.min())
     
    svd_profile = pd.DataFrame(svd_profile,index = playlist_index ,columns = columns_name).transpose()
    
    return svd_profile

In [86]:
playlist_id_t = df_tracks["playlist_id"].unique()

In [87]:
svd_mat = construct_svd_df(df_svd,playlist_id_t)
svd_mat

(471, 28356)


,37i9dQZF1DXcZDD7cfEKhW,37i9dQZF1DWZQaaqNMbbXa,37i9dQZF1DX2ENAPP1Tyed,37i9dQZF1DWSJHnPb1f0X3,37i9dQZF1DX6pH08wMhkaI,37i9dQZF1DX3PIAZMcbo2T,37i9dQZF1DXahYFr91pFvG,37i9dQZF1DX4RDXswvP6Mj,0B2HdP15IucgE0vk3sluJR,4SdfG4cPG3skmTiQLozZGh,...,0MhTMIo1bgH6zzPh7BdChT,4IS7o1utOzhimFEFnj9gmu,2dNitDEHM9FpUGEHWc7zyW,0o5gKmuET7F3lQnpFxvPO8,5BqOZpVEqRDfZScvW1QUyA,0FCHg9zJMNNiOokh3hVcxd,73uj4YmsC7SJ6SbUMTvf07,29jj7pQlDqnWclbHQk21Rq,4N1ipiKR3xla8UXtE12XBm,6jI1gFr6ANFtT8MmTvA2Ux
track_id,,,,,,,,,,,,,,,,,,,,,
0017A6SJgTbfQVU2EtsPNo,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082,...,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082,0.076082
002xjHwzEx66OWFV2IP9dk,0.074357,0.075900,0.076124,0.076008,0.075613,0.074983,0.070547,0.078506,0.077336,0.075155,...,0.074256,0.076960,0.071683,0.075532,0.075046,0.076191,0.073864,0.076917,0.076468,0.074857
004s3t0ONYlzxII9PLgU6z,0.068756,0.076083,0.076045,0.076072,0.076147,0.077687,0.076141,0.076252,0.072845,0.076085,...,0.076007,0.076695,0.077354,0.073881,0.073309,0.076087,0.077800,0.076242,0.076060,0.076225
008MceT31RotUANsKuzy3L,0.077732,0.076161,0.076218,0.076098,0.076102,0.075714,0.075556,0.076743,0.076478,0.076806,...,0.076467,0.076024,0.075591,0.076116,0.076102,0.076097,0.076363,0.076564,0.076068,0.076183
008rk8F6ZxspZT4bUlkIQG,0.076073,0.076083,0.076083,0.076082,0.076086,0.076077,0.076098,0.076067,0.076084,0.076080,...,0.076100,0.076093,0.076078,0.076082,0.076084,0.076082,0.076095,0.076083,0.076081,0.076082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7zxRMhXxJMQCeDDg0rKAVo,0.070634,0.076101,0.076388,0.076009,0.075182,0.076959,0.076546,0.076825,0.073477,0.076109,...,0.078932,0.074218,0.074544,0.076098,0.076038,0.076059,0.081221,0.076224,0.075730,0.076161
7zyLObYw4QUKQDyZOb4J0Y,0.074318,0.076081,0.076071,0.076083,0.075252,0.092382,0.076311,0.076028,0.080404,0.076112,...,0.076271,0.076204,0.074239,0.076113,0.076097,0.076087,0.076108,0.076052,0.076078,0.076132
7zycSpvjDcqh6YT1FEl2kY,0.073168,0.076172,0.076201,0.076110,0.075902,0.076306,0.076210,0.076188,0.075142,0.076454,...,0.076788,0.076392,0.076079,0.076114,0.076018,0.076082,0.078296,0.076386,0.075981,0.076184


# We then incorporate the SVD into our recommender.

In [88]:
class SVDRecommender:
    
      def __init__(self,df_tracks,svd_mat):
            
         self.df_tracks = df_tracks
        
         self.svd_mat = svd_mat
          
      def recommend(self,playlist_id,is_exempt = True):
        
          self.df_tracks.set_index("playlist_id")
                                   
          if not is_exempt:
     
             exempt_id = []
                                   
          else:  
                                   
             exempt_id = self.df_tracks.loc[self.df_tracks["playlist_id"] == playlist_id]["track_id"].to_list()
      
          res = self.svd_mat[playlist_id].sort_values(ascending = False).reset_index().rename(columns={playlist_id: 'recs'})
      
          res = res[~res.track_id.isin(exempt_id)]
        
          recommend_track = list()
            
          for t_id,rec in zip(res["track_id"],res["recs"]):
              
              recommend_track.append((t_id,rec))
            
          return recommend_track

In [89]:
svd_collaborate = SVDRecommender(df_tracks,svd_mat)

In [90]:
playlist_id = "37i9dQZF1DX2ENAPP1Tyed"
recommend_list = svd_collaborate.recommend(playlist_id)
recommend_list = pd.DataFrame(recommend_list,columns=["track_id","intensity"])
recommendations = pd.merge(recommend_list,df_tracks.drop_duplicates(subset='track_id', keep="first"), how='left', on='track_id')
recommendations[['track_id', 'track_name', 'artist', 'album', 'intensity']].head()

,track_id,track_name,artist,album,intensity
0,0fea68AdmYNygeTGI4RC18,LA CANCIÓN,J Balvin,OASIS,0.108261
1,2TH65lNHgvLxCKXM3apjxI,Callaita,Bad Bunny,Callaita,0.107160
2,1c0hsvHLELX6y8qymnpLKL,Soltera - Remix,Lunay,Soltera (Remix),0.107143
3,37zdqI4r1gswIzczSBkRon,Aventura,Lunay,Épico,0.105516
4,4bTZeO72FwMa6wKOiqoynL,Otro Trago - Remix,Sech,Otro Trago (Remix),0.104824


In [91]:
evaluate(df_tracks,svd_collaborate,N=100)

{'recall@5': 0.043758542641905145, 'recall@10': 0.08949997907832076, 'recall@20': 0.17975788703746712}


# Song Features Collaborative Recommender

The recommender first pre-calculate track cosine similarity matrix based on attributes like 'danceability', 'energy', 'key', 'loudness','mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness' etc as we conjectured that songs in the same playlist share similar attributes for them to be put on the same playlist. Then for each target playlist id, it returns a list of tracks (prediction list) to recommend per playlist. The prediction list contains top k similar songs (based on cosine similarity) per track in the playlist.

In [92]:
class Song_Feature_CollaborativeRecommender:
      
      def __init__(self,df_tracks,features_drop):
     
          self.df_tracks = df_tracks
        
          self.features_drop = features_drop
            
          self.feature_similarity = None
        
          self.df_tracks_cleaned = None
            
      def scaled_columns(self):
        
          df_tracks1 = self.df_tracks.copy()
            
          df_tracks1 = df_tracks1.fillna(0)        
        
          df_tracks_cleaned = df_tracks1.drop(columns = self.features_drop)
            
          exempt_cols = ["playlist_id","track_id"]
                         
          for colname in df_tracks_cleaned.columns:
      
              if colname in exempt_cols:
            
                 continue
                
              new_col_name = "scaled_"+ colname
      
              print(new_col_name)
        
              df_tracks_cleaned[new_col_name] = (df_tracks_cleaned[colname] - min(df_tracks_cleaned[colname])) / (max(df_tracks_cleaned[colname]) - min(df_tracks_cleaned[colname]))
       
              df_tracks_cleaned = df_tracks_cleaned.drop(columns = colname)
        
              df_tracks_cleaned1 = df_tracks_cleaned.copy()
            
              self.df_tracks_cleaned = df_tracks_cleaned1.drop_duplicates().reset_index() 
                
                
      def calculate_similarity(self):
        
          self.feature_similarity = cosine_similarity(self.df_tracks_cleaned.drop(columns = {'track_id','playlist_id'}))
            
      def recommend(self,playlist_id,is_exempt = True):
        
          target_track_id = self.df_tracks_cleaned[self.df_tracks_cleaned["playlist_id"]==playlist_id]["track_id"].tolist()
        
          strength = list()
            
          if len(target_track_id)<10:
            
             return [],-1
      
          train,test = train_test_split(target_track_id, test_size = 0.2, random_state=0)
           
          for i,track_id in enumerate(train):
            
              playlist_id_index =  self.df_tracks_cleaned[self.df_tracks_cleaned["playlist_id"]==playlist_id].index
     
              tracks_id_index =  self.df_tracks_cleaned[self.df_tracks_cleaned["track_id"]==track_id].index
      
              track_index = list(set(tracks_id_index).intersection(set(playlist_id_index)))
                    
              track_index = track_index[0]
            
              similar_value = self.feature_similarity[track_index,:]
                
              similar_index = np.argsort(similar_value)
            
              sorted_values = np.sort(similar_value)[::-1]
                
              tracks_id_list = self.df_tracks_cleaned.iloc[similar_index]["track_id"][::-1].drop_duplicates()
                       
              candidate_id_list = np.array(tracks_id_list[~tracks_id_list.isin(tracks_id_index)])
                
              rec_list = candidate_id_list[:6]
               
              strength.extend(sorted_values[:6])            
    
              if i == 0:
            
                 res_list = rec_list
                 
              else:
                
                 res_list = np.append(res_list, rec_list)
                    
          
      
          recommend_track = list()
      
          res_list = list(res_list)
                     
          for track,intensity in zip(res_list,strength):
     
              recommend_track.append((track,intensity))
      
          return recommend_track,test  

Evaluation metric used in our song features collaborative recommender.

In [93]:
import random
from sklearn.model_selection import train_test_split

def evaluate_song_playlist(df_tracks,model,N=450):
    
    playlists = df_tracks["playlist_id"].unique()
                          
    playlist_metric = list()
                          
    precisions = 0
    
    total = 0
                          
    for _,playlist in enumerate(playlists):
                          
        precision = 0
           
        recommend_list,test = model.recommend(playlist)
        
        if recommend_list == []:
            
           continue
        
        else:
        
           total += 1
                          
        recommend_ids = list(list(zip(*recommend_list))[0])
                          
        overlap = len(set(recommend_ids).intersection(set(test)))
                          
        precision += overlap/len(test)
                                                     
        playlist_metric.append((precision,1)) 
                          
    total_tests = sum(list(list(zip(*playlist_metric))[1]))
      
    precision = sum(list(list(zip(*playlist_metric))[0])) / total
      
    recall = {'hit@100': precision}                
    
    print(recall)

In [94]:
df_tracks.columns

Index(['track_id', 'track_name', 'artist', 'popularity', 'album_id', 'album',
       'track_album_release_date', 'playlist_name', 'playlist_id', 'genre',
       'playlist_subgenre', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'full_description', 'exist'],
      dtype='object')

In [95]:
features_drop = ['track_name', 'artist','album_id', 'album','track_album_release_date', 'playlist_name', 'genre','playlist_subgenre','full_description','exist']

In [96]:
#Song_Collaborator = Song_Content_CollaborativeRecommender(df_tracks[:35000],features_drop)
Song_Collaborator = Song_Feature_CollaborativeRecommender(df_tracks,features_drop)

In [97]:
Song_Collaborator.scaled_columns()

scaled_popularity
scaled_danceability
scaled_energy
scaled_key
scaled_loudness
scaled_mode
scaled_speechiness
scaled_acousticness
scaled_instrumentalness
scaled_liveness
scaled_valence
scaled_tempo
scaled_duration_ms


In [98]:
Song_Collaborator.df_tracks_cleaned.index

RangeIndex(start=0, stop=32251, step=1)

In [99]:
Song_Collaborator.calculate_similarity()

In [100]:
evaluate_song_playlist(Song_Collaborator.df_tracks_cleaned,Song_Collaborator)

{'hit@100': 0.30810522753558506}


# Neural Collaborative Filtering with NeuMF
I also implement the Collaborative Filtering using neural network approach from [He et al, 2017,](https://arxiv.org/pdf/1708.05031.pdf) Steps are as follow:
- Build sparse one-hot encoding matrix between users (playlists) and items (tracks).
- Playlist and songs tracks vectors are used to create embeddings (low-dimensional) for each playlist and item.
- Generalized Matrix Factorization (GMF) combines the two embeddings using dot product.
- Multi-layer perceptron (MLP) can also create embeddings for playlist and tracks by passing both the playlists and tracks   embeddings to deeper layers
- NeuMF then combines the predictions from MLP and GMF to obtain the recommendation.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralCollaborateRecommender(nn.Module):
    
      def __init__(self,num_users,num_items,embed_dim):
            
         super(NeuralCollaborateRecommender, self).__init__()
        
         self.num_users = num_users
            
         self.num_items = num_items
        
         self.mlp_user_embed = nn.Embedding(self.num_users,embed_dim)
            
         self.mlp_item_embed = nn.Embedding(self.num_items,embed_dim)
        
         self.mf_user_embed = nn.Embedding(self.num_users,embed_dim)
            
         self.mf_item_embed = nn.Embedding(self.num_items,embed_dim)
        
         self.mlp_encoder = nn.Sequential(
             
              nn.Linear(2*embed_dim,embed_dim),               
              nn.ReLU(),             
              nn.Linear(embed_dim,embed_dim//2), 
              nn.ReLU(),
              nn.Linear(embed_dim//2,embed_dim//4), 
              nn.ReLU(),
              nn.Linear(embed_dim//4,embed_dim//8)
              
         )
            
         self.neural_encoder = nn.Linear(embed_dim//8+ embed_dim,1)
        
         self.predict = nn.Sigmoid()
          
      def forward(self,user_input,item_input,label):
      
          user_input = torch.from_numpy(user_input)
      
          item_input = torch.from_numpy(item_input)
        
          label = torch.from_numpy(label)
            
          u_mlp_embed = self.mlp_user_embed(user_input)
        
          i_mlp_embed = self.mlp_item_embed(item_input)
            
          u_mf_embed = self.mf_user_embed(user_input)
        
          i_mf_embed = self.mf_item_embed(item_input)
            
          mf_input = u_mf_embed * i_mf_embed
        
          mlp_input = torch.cat([u_mlp_embed,i_mlp_embed], dim = -1)
            
          mlp_out = self.mlp_encoder(mlp_input)
        
          m_neural_in = torch.cat([mlp_out,mf_input], dim = -1)
            
          y_predict = self.neural_encoder(m_neural_in)
        
          y_predict = self.predict(y_predict)
            
          print(y_predict)
        
          loss = F.binary_cross_entropy(y_predict.squeeze(),label.float()) 
            
          loss = loss.mean()
        
          return loss,y_predict

In [ ]:
df_svd

In [148]:
from collections import defaultdict

In [149]:
df_svd_val = df_svd.values
num_users,num_items = df_svd_val.shape
print(num_items,num_users)

28356 471


In [150]:
num_negatives = 4

In [151]:
users,items  = np.nonzero(df_svd_val)

In [152]:
users.size/(28356*471)

0.002414778555574424

In [153]:
interaction_list = defaultdict(list)
interaction_set = defaultdict(set)
last_interaction = dict()

In [154]:
for u,i in zip(users,items):
    interaction_list[u].append(i)
    interaction_set[u].add(i)
for u in interaction_list.keys():
    last_interaction[u] = interaction_list[u][len(interaction_list[u])-1]

In [155]:
occurence = defaultdict(set)

In [156]:
user_index,item_index,labels = list(),list(),list()

In [157]:
for u in interaction_list.keys():
    
    for i in range(len(interaction_list[u])-1):
        
        user_index.append(u)
        
        item_index.append(interaction_list[u][i])
        
        labels.append(1)
        
        for neg in range(num_negatives):
            
            item_idx = np.random.randint(num_items,size = 1)[0]
            
            while item_idx in interaction_set[u] or item_idx in occurence[u]:
                
                  item_idx = np.random.randint(num_items,size = 1)[0]
                    
            user_index.append(u)
            
            item_index.append(item_idx)
            
            labels.append(0)
            
            occurence[u].add(item_idx)
            
test_user_index,test_item_index,test_labels = list(), list(),list()

test_index = defaultdict(list)

index_count  = 0

test_occurence = defaultdict(set)

for u in interaction_list.keys():
    
    i = last_interaction[u]
    
    test_user_index.append(u)
    
    test_item_index.append(i)
    
    test_labels.append(1)
    
    test_index[u].append(index_count)
    
    index_count += 1
    
    for neg in range(40):
        
        item_idx = np.random.randint(num_items,size = 1)[0]
        
        while item_idx in interaction_set[u] or item_idx in test_occurence[u]:
                
              item_idx = np.random.randint(num_items,size = 1)[0]
            
        test_user_index.append(u)
            
        test_item_index.append(item_idx)
            
        test_labels.append(0)
            
        test_occurence[u].add(item_idx)  
        
        test_index[u].append(index_count)
    
        index_count += 1
         

In [158]:
neural_collab = NeuralCollaborateRecommender(num_users,num_items,64)

In [159]:
optimizer = torch.optim.Adam(neural_collab.parameters(),lr=0.0015, eps=1e-4)

In [160]:
for epoch in range(5000):
     
    loss,y_predict = neural_collab(np.array(user_index),np.array(item_index),np.array(labels))
    
    print(loss)
    
    optimizer.zero_grad() #reset gradients to 0\n",
     
    loss.backward(retain_graph=False) #this calculates the gradients\n",
     
    optimizer.step()

tensor([[0.4659],
        [0.3076],
        [0.6194],
        ...,
        [0.3916],
        [0.4530],
        [0.4763]], grad_fn=<SigmoidBackward0>)
tensor(0.6919, grad_fn=<MeanBackward0>)
tensor([[0.4649],
        [0.3116],
        [0.6137],
        ...,
        [0.3947],
        [0.4519],
        [0.4742]], grad_fn=<SigmoidBackward0>)
tensor(0.6891, grad_fn=<MeanBackward0>)
tensor([[0.4638],
        [0.3157],
        [0.6080],
        ...,
        [0.3978],
        [0.4508],
        [0.4721]], grad_fn=<SigmoidBackward0>)
tensor(0.6863, grad_fn=<MeanBackward0>)
tensor([[0.4628],
        [0.3198],
        [0.6026],
        ...,
        [0.4009],
        [0.4497],
        [0.4701]], grad_fn=<SigmoidBackward0>)
tensor(0.6837, grad_fn=<MeanBackward0>)
tensor([[0.4618],
        [0.3240],
        [0.5971],
        ...,
        [0.4039],
        [0.4487],
        [0.4682]], grad_fn=<SigmoidBackward0>)
tensor(0.6810, grad_fn=<MeanBackward0>)
tensor([[0.4607],
        [0.3282],
        [0.591

tensor([[0.1525],
        [0.1476],
        [0.1612],
        ...,
        [0.2483],
        [0.2354],
        [0.2324]], grad_fn=<SigmoidBackward0>)
tensor(0.5050, grad_fn=<MeanBackward0>)
tensor([[0.1373],
        [0.1326],
        [0.1435],
        ...,
        [0.2333],
        [0.2208],
        [0.2178]], grad_fn=<SigmoidBackward0>)
tensor(0.5058, grad_fn=<MeanBackward0>)
tensor([[0.1249],
        [0.1202],
        [0.1292],
        ...,
        [0.2196],
        [0.2075],
        [0.2046]], grad_fn=<SigmoidBackward0>)
tensor(0.5077, grad_fn=<MeanBackward0>)
tensor([[0.1157],
        [0.1109],
        [0.1188],
        ...,
        [0.2080],
        [0.1961],
        [0.1934]], grad_fn=<SigmoidBackward0>)
tensor(0.5099, grad_fn=<MeanBackward0>)
tensor([[0.1098],
        [0.1049],
        [0.1122],
        ...,
        [0.1987],
        [0.1871],
        [0.1845]], grad_fn=<SigmoidBackward0>)
tensor(0.5118, grad_fn=<MeanBackward0>)
tensor([[0.1069],
        [0.1018],
        [0.109

tensor([[0.1751],
        [0.1599],
        [0.1872],
        ...,
        [0.1794],
        [0.1643],
        [0.1757]], grad_fn=<SigmoidBackward0>)
tensor(0.5000, grad_fn=<MeanBackward0>)
tensor([[0.1768],
        [0.1617],
        [0.1889],
        ...,
        [0.1808],
        [0.1658],
        [0.1773]], grad_fn=<SigmoidBackward0>)
tensor(0.5000, grad_fn=<MeanBackward0>)
tensor([[0.1784],
        [0.1634],
        [0.1906],
        ...,
        [0.1823],
        [0.1672],
        [0.1788]], grad_fn=<SigmoidBackward0>)
tensor(0.4999, grad_fn=<MeanBackward0>)
tensor([[0.1800],
        [0.1650],
        [0.1923],
        ...,
        [0.1837],
        [0.1686],
        [0.1803]], grad_fn=<SigmoidBackward0>)
tensor(0.4998, grad_fn=<MeanBackward0>)
tensor([[0.1814],
        [0.1664],
        [0.1938],
        ...,
        [0.1849],
        [0.1699],
        [0.1816]], grad_fn=<SigmoidBackward0>)
tensor(0.4998, grad_fn=<MeanBackward0>)
tensor([[0.1826],
        [0.1677],
        [0.195

tensor([[0.1777],
        [0.1640],
        [0.1903],
        ...,
        [0.1811],
        [0.1649],
        [0.1752]], grad_fn=<SigmoidBackward0>)
tensor(0.4987, grad_fn=<MeanBackward0>)
tensor([[0.1775],
        [0.1638],
        [0.1901],
        ...,
        [0.1809],
        [0.1646],
        [0.1748]], grad_fn=<SigmoidBackward0>)
tensor(0.4987, grad_fn=<MeanBackward0>)
tensor([[0.1773],
        [0.1636],
        [0.1899],
        ...,
        [0.1807],
        [0.1643],
        [0.1745]], grad_fn=<SigmoidBackward0>)
tensor(0.4987, grad_fn=<MeanBackward0>)
tensor([[0.1771],
        [0.1634],
        [0.1897],
        ...,
        [0.1805],
        [0.1640],
        [0.1741]], grad_fn=<SigmoidBackward0>)
tensor(0.4987, grad_fn=<MeanBackward0>)
tensor([[0.1769],
        [0.1632],
        [0.1896],
        ...,
        [0.1803],
        [0.1638],
        [0.1738]], grad_fn=<SigmoidBackward0>)
tensor(0.4986, grad_fn=<MeanBackward0>)
tensor([[0.1767],
        [0.1630],
        [0.189

tensor([[0.1768],
        [0.1634],
        [0.1910],
        ...,
        [0.1807],
        [0.1612],
        [0.1690]], grad_fn=<SigmoidBackward0>)
tensor(0.4979, grad_fn=<MeanBackward0>)
tensor([[0.1768],
        [0.1635],
        [0.1910],
        ...,
        [0.1808],
        [0.1612],
        [0.1689]], grad_fn=<SigmoidBackward0>)
tensor(0.4979, grad_fn=<MeanBackward0>)
tensor([[0.1769],
        [0.1635],
        [0.1910],
        ...,
        [0.1807],
        [0.1611],
        [0.1688]], grad_fn=<SigmoidBackward0>)
tensor(0.4979, grad_fn=<MeanBackward0>)
tensor([[0.1769],
        [0.1636],
        [0.1910],
        ...,
        [0.1807],
        [0.1611],
        [0.1687]], grad_fn=<SigmoidBackward0>)
tensor(0.4978, grad_fn=<MeanBackward0>)
tensor([[0.1769],
        [0.1636],
        [0.1911],
        ...,
        [0.1807],
        [0.1610],
        [0.1686]], grad_fn=<SigmoidBackward0>)
tensor(0.4978, grad_fn=<MeanBackward0>)
tensor([[0.1769],
        [0.1637],
        [0.191

...

tensor([[7.7758e-01],
        [7.9458e-03],
        [2.6769e-04],
        ...,
        [1.2617e-02],
        [6.5554e-03],
        [2.2229e-03]], grad_fn=<SigmoidBackward0>)
tensor(0.1530, grad_fn=<MeanBackward0>)
tensor([[7.7814e-01],
        [7.8556e-03],
        [2.5802e-04],
        ...,
        [1.2610e-02],
        [6.4261e-03],
        [2.1731e-03]], grad_fn=<SigmoidBackward0>)
tensor(0.1525, grad_fn=<MeanBackward0>)
tensor([[7.7869e-01],
        [7.7522e-03],
        [2.4892e-04],
        ...,
        [1.2632e-02],
        [6.3032e-03],
        [2.1262e-03]], grad_fn=<SigmoidBackward0>)
tensor(0.1520, grad_fn=<MeanBackward0>)
tensor([[7.7954e-01],
        [7.6492e-03],
        [2.4064e-04],
        ...,
        [1.2617e-02],
        [6.1941e-03],
        [2.0827e-03]], grad_fn=<SigmoidBackward0>)
tensor(0.1515, grad_fn=<MeanBackward0>)
tensor([[7.7960e-01],
        [7.5399e-03],
        [2.3384e-04],
        ...,
        [1.2533e-02],
        [6.0910e-03],
        [2.0415e-03]]

tensor([[8.2406e-01],
        [3.9681e-03],
        [1.0633e-04],
        ...,
        [7.7742e-03],
        [3.3709e-03],
        [1.1832e-03]], grad_fn=<SigmoidBackward0>)
tensor(0.1371, grad_fn=<MeanBackward0>)
tensor([[8.3694e-01],
        [4.1571e-03],
        [1.1052e-04],
        ...,
        [8.1560e-03],
        [3.6572e-03],
        [1.2672e-03]], grad_fn=<SigmoidBackward0>)
tensor(0.1367, grad_fn=<MeanBackward0>)
tensor([[8.3759e-01],
        [4.0473e-03],
        [1.0818e-04],
        ...,
        [8.0246e-03],
        [3.6158e-03],
        [1.2484e-03]], grad_fn=<SigmoidBackward0>)
tensor(0.1364, grad_fn=<MeanBackward0>)
tensor([[8.2815e-01],
        [3.7093e-03],
        [1.0077e-04],
        ...,
        [7.4663e-03],
        [3.2980e-03],
        [1.1472e-03]], grad_fn=<SigmoidBackward0>)
tensor(0.1361, grad_fn=<MeanBackward0>)
tensor([[8.3917e-01],
        [3.8910e-03],
        [1.0383e-04],
        ...,
        [7.7366e-03],
        [3.4994e-03],
        [1.2161e-03]]

tensor([[8.9387e-01],
        [2.6006e-03],
        [5.1579e-05],
        ...,
        [4.6970e-03],
        [1.9520e-03],
        [8.5200e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1243, grad_fn=<MeanBackward0>)
tensor([[8.9327e-01],
        [2.5666e-03],
        [5.0490e-05],
        ...,
        [4.5818e-03],
        [1.8904e-03],
        [8.3443e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1240, grad_fn=<MeanBackward0>)
tensor([[8.9428e-01],
        [2.5765e-03],
        [5.0050e-05],
        ...,
        [4.5217e-03],
        [1.8647e-03],
        [8.2940e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1237, grad_fn=<MeanBackward0>)
tensor([[8.9500e-01],
        [2.5790e-03],
        [4.9366e-05],
        ...,
        [4.4550e-03],
        [1.8323e-03],
        [8.2053e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1234, grad_fn=<MeanBackward0>)
tensor([[8.9469e-01],
        [2.5433e-03],
        [4.8137e-05],
        ...,
        [4.3667e-03],
        [1.7794e-03],
        [8.0039e-04]]

tensor([[9.0519e-01],
        [1.8920e-03],
        [2.8267e-05],
        ...,
        [3.0563e-03],
        [1.0262e-03],
        [5.5992e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1129, grad_fn=<MeanBackward0>)
tensor([[9.0620e-01],
        [1.8896e-03],
        [2.7984e-05],
        ...,
        [3.0368e-03],
        [1.0170e-03],
        [5.5688e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1126, grad_fn=<MeanBackward0>)
tensor([[9.0647e-01],
        [1.8745e-03],
        [2.7554e-05],
        ...,
        [2.9972e-03],
        [9.9356e-04],
        [5.4878e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1123, grad_fn=<MeanBackward0>)
tensor([[9.0700e-01],
        [1.8645e-03],
        [2.7389e-05],
        ...,
        [2.9777e-03],
        [9.8233e-04],
        [5.4657e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1120, grad_fn=<MeanBackward0>)
tensor([[9.0637e-01],
        [1.8349e-03],
        [2.6845e-05],
        ...,
        [2.9138e-03],
        [9.5434e-04],
        [5.3608e-04]]

tensor([[9.2242e-01],
        [1.4307e-03],
        [1.7273e-05],
        ...,
        [2.0026e-03],
        [4.8318e-04],
        [3.3494e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1024, grad_fn=<MeanBackward0>)
tensor([[9.2722e-01],
        [1.4573e-03],
        [1.7697e-05],
        ...,
        [2.0318e-03],
        [5.0171e-04],
        [3.4602e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1022, grad_fn=<MeanBackward0>)
tensor([[9.2365e-01],
        [1.3819e-03],
        [1.6623e-05],
        ...,
        [1.9331e-03],
        [4.5030e-04],
        [3.1996e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1019, grad_fn=<MeanBackward0>)
tensor([[9.3300e-01],
        [1.4589e-03],
        [1.7453e-05],
        ...,
        [2.0102e-03],
        [5.0038e-04],
        [3.4601e-04]], grad_fn=<SigmoidBackward0>)
tensor(0.1017, grad_fn=<MeanBackward0>)
tensor([[9.2315e-01],
        [1.3271e-03],
        [1.5747e-05],
        ...,
        [1.8538e-03],
        [4.1806e-04],
        [3.0103e-04]]

In [161]:
test_loss,test_predict = neural_collab(np.array(test_user_index),np.array(test_item_index),np.array(test_labels))

tensor([[4.5862e-04],
        [9.9999e-01],
        [2.3988e-05],
        ...,
        [3.0417e-01],
        [8.7557e-01],
        [9.6710e-02]], grad_fn=<SigmoidBackward0>)


In [166]:
hit = 0 

for user in test_index.keys():

    index = torch.argsort(test_predict[test_index[user]].squeeze(),descending=True)[:10]
    
    if 0 in index:
     
       hit += 1
hit    

92